# Plain Causal Simulator

This notebook fulfills the following:

* a simple method for fitting a bog-standard (binary) treatment model with pre-treatment covariates. One of its uses to generate a gold standard model (by fitting it to real data) where parameters are not unrealistic.
* a method for simulating for those models
* a way of computing the corresponding CATEs from a fitted model
* a way of doing column selection for introducing unmeasured confounding by dropping a subset of columns out of simulated data, so that the corresponding model should feel the impact of unmeasured confounding
* a purely-random generator of causal models by sampling parameters.
* illustrative examples

Not done yet: `learn_backdoor_model` is appropriate for creating a simulated ground truth, but for comparisons of different methods I'd use a more specialised CATE fitting method since `learn_backdoor_model` is not targeted to that task specifically.

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import copy

from dataclasses import dataclass
from multipledispatch import dispatch
from scipy.stats import invwishart, spearmanr
from sklearn.ensemble import IsolationForest
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import FastICA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

---

### General Utilities

General utilities not exclusive for the main goals of causal model generation and evaluation.

In [2]:
############ Utilities in general

# Converts columns in a DataFrame to categorical where the number of unique values
# is less than or equal to `cat_threshold`.

def dataframe_categorize(df, cat_threshold):
  for column in df.columns:
    if df[column].nunique() <= cat_threshold:
      df[column] = df[column].astype(int).astype('category')

# The following was generated by Copilot
def cov_to_corr(cov_matrix):
  """
  Converts a covariance matrix to a correlation matrix.

  Parameters:
      cov_matrix (numpy.ndarray): Covariance matrix.

  Returns:
      numpy.ndarray: Correlation matrix.
  """
  std_dev = np.sqrt(np.diag(cov_matrix))
  outer_std_dev = np.outer(std_dev, std_dev)
  corr_matrix = cov_matrix / outer_std_dev
  corr_matrix[np.diag_indices_from(corr_matrix)] = 1    
  return corr_matrix

# Logistic function
def logistic(x):
    return 1 / (1 + np.exp(-x))

# Learn a XGBoost (binary) classifier or regression function.
# 

def learn_xgb(input_dat, output_dat):

  param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2]
  }

  if isinstance(output_dat.dtype, pd.CategoricalDtype):
    bst = xgb.XGBClassifier(objective='binary:logistic', enable_categorical=True)
  else:
    bst = xgb.XGBRegressor(objective='reg:squarederror', enable_categorical=True)

  grid_search = GridSearchCV(estimator=bst, param_grid=param_grid, cv=3, scoring='accuracy', verbose=0)
  grid_search.fit(input_dat, output_dat)
  bst = grid_search.best_estimator_

  return bst

# Predict expected value of output of a XGBoost model `bst` based on
# input `x_val` and whether output is categorical (`y_dtype`). Only binary
# and continuous outcomes are allowed.

def predict_xgb(bst, x_eval, y_dtype):
  if isinstance(y_dtype, pd.CategoricalDtype):
    return bst.predict_proba(x_eval)[:, 1]
  return bst.predict(x_eval)

# Print a confusion matrix

def print_confusion_matrix(y_true, y_pred, labels=None):
  cm = confusion_matrix(y_true, y_pred, labels=labels)
  cm_df = pd.DataFrame(cm, index=labels, columns=labels)
  print('Confusion Matrix:')
  print(cm_df)


---

### Parameter Learning, Simulation and Ground Truth

The following piece of code fits data to a classic causal model that is fully defined by a triplet: a (binary) treatment variable $A$, some (binary or continuous) outcome $Y$, given pre-treatment covariates $X$. Conditional ignorability is assumed.

There is also code for simulating from a fitted model, and a method to generate CATEs from a given model. The latter means that if a model is the ground truth one, the corresponding CATEs are ground truth. 

Given a model, we also have a method (`confounder_column_keeper`) to select a subset of columns to keep, while hiding others, in order to generate data from a model which violated conditional ignorability.

We use `pandas.DataFrame` as our main data structure to facilitate the use of categorical data in `xgboost`.

In [3]:
# Class BackdoorModel:
#
# This stores structural information for models composed on synthetic models, which are based on (correlated) standard Gaussians
# as covariates, with treatment and outcome being binary variables generated by a logistic model.

@dataclass
class BackdoorModel:
  x_idx: str                   # Column names of covariates
  a_idx: str                   # Column name of treatment
  y_idx: str                   # Column name of outcome

# Class XGBoostBackdoorModel:
#
# This stores information for models fit to data, based on XGBoost for treatment and outcome,
# and a plain empirical distribution for the covariates. 

@dataclass
class XGBoostBackdoorModel(BackdoorModel):
  bst_propensity: xgb.XGBModel # Propensity score model
  bst_outcome: xgb.XGBModel    # Propensity score model
  std_outcome: float           # Standard deviation of outcome regression, if applicable
  empirical_df: pd.DataFrame   # Empirical distribution for generating covariates     

# Parameter learning of basic causal model with triplet (treatment, outcome, pre_treatment covariates).
# 
# This method adopts no model for covariates (we will use the empirical distribution). It assumes
# a binary treatment, and either a binary or continuous outcome. It uses XGBoost for conditional 
# means and, in case out a continuous outcome, a homescedastic Gaussian error model.
#
# Inputs:
#
# - `train`, a DataFrame wit the data 
# - `x_idx`, names covariate columns
# - `a_idx`, name of treatment column
# - `y_idx`, name of outcome column
#
# This returns the model for propensities, the model for the outcome, and if outcome
# is continuous, the standard deviation of the error term, all wrapped up in
# a `XGBoostBackdoorModel` object.
#
# Call this method if you want to fit a generative model. If all you wish is a CATE
# estimator, use something else like estimate_cate_tlearner.

def learn_backdoor_model(train, x_idx, a_idx, y_idx):
  
  x_train, a_train, y_train = train[x_idx], train[a_idx], train[y_idx]
  ax_train = pd.concat([a_train, x_train], axis=1)

  bst_propensity = learn_xgb(x_train, a_train)
  bst_outcome = learn_xgb(ax_train, y_train)

  if isinstance(y_train.dtype, pd.CategoricalDtype):
    std_outcome = None
  else:
    std_outcome = np.std(bst_outcome.predict(ax_train))

  return XGBoostBackdoorModel(bst_propensity=bst_propensity, bst_outcome=bst_outcome, std_outcome=std_outcome,
                              x_idx=x_idx, a_idx=a_idx, y_idx=y_idx, empirical_df=train)

# Simulation from a fitted model.
#
# Basic simulation out of a causal model based on a XGBoost fit. Covariates are generated
# from a boring model-free bootstrap sample out of `model.empirical_pd`.
#
# Inputs:
#
# - `n` is the number of samples
# - `model` is a fitted model of the type `XBoostBackdoorModel`

@dispatch(int, XGBoostBackdoorModel)
def simulate_from_backdoor_model(n, model):
  row_choices = np.random.choice(range(model.empirical_df.shape[0]), size=n)
  df_sample = model.empirical_df.iloc[row_choices, :].reset_index(drop=True)
  propensities = model.bst_propensity.predict_proba(df_sample[model.x_idx])[:, 1]
  df_sample[model.a_idx] = np.random.binomial(n=1, p=propensities)
  df_sample[model.a_idx] = df_sample[model.a_idx].astype('category') # Somehow, this is necessary

  e_y = predict_xgb(model.bst_outcome, df_sample[[*[model.a_idx], *model.x_idx]], model.empirical_df[model.y_idx].dtype)
  if model.std_outcome == None:
    df_sample[model.y_idx] = np.random.binomial(n=1, p=e_y)
    df_sample[model.y_idx] = df_sample[model.y_idx].astype('category') # Somehow, this is necessary
  else:
    df_sample[model.y_idx] = e_y + np.random.normal(loc=0, scale=model.std_outcome, size=n)

  return df_sample

@dispatch(int, XGBoostBackdoorModel)
def simulate_from_controlled_backdoor_model(n, model):
  row_choices = np.random.choice(range(model.empirical_df.shape[0]), size=n)
  df_sample = model.empirical_df.iloc[row_choices, :].reset_index(drop=True)
  df_sample[model.a_idx] = np.random.binomial(n=1, p=0.5 * np.ones(n))
  df_sample[model.a_idx] = df_sample[model.a_idx].astype('category') # Somehow, this is necessary

  e_y = predict_xgb(model.bst_outcome, df_sample[[*[model.a_idx], *model.x_idx]], model.empirical_df[model.y_idx].dtype)
  if model.std_outcome == None:
    df_sample[model.y_idx] = np.random.binomial(n=1, p=e_y)
    df_sample[model.y_idx] = df_sample[model.y_idx].astype('category') # Somehow, this is necessary
  else:
    df_sample[model.y_idx] = e_y + np.random.normal(loc=0, scale=model.std_outcome, size=n)

  return df_sample

# Selection of observable columns.
#
# This decides which covariates to keep so that everything else will count as unmeasured 
# # confounding. We will use XGBoost's built-in measure of variable importance to decide 
# which variables to be throw out.
#
# Inputs:
#
# - `keep_k`: how many of the covariates to keep as observable.
# - `model`: the corresponding `XGBoostBackdoorModel` model.

@dispatch(int, XGBoostBackdoorModel)
def confounder_column_keeper(keep_k, model):
  keep_x = np.argsort(model.bst_propensity.feature_importances_)[-keep_k:]
  x_select = [model.x_idx[i] for i in keep_x]
  return x_select

# Compute CATE from given model.
#
# Inputs:
#
# - `df_sample` is the table of covariates we evaluate at
# - `model` is the corresponding `XGBoostBackdoorModel` fitted model.

@dispatch(pd.DataFrame, XGBoostBackdoorModel)
def get_cate(df_sample, model):
  
  n = df_sample.shape[0]
  a = pd.DataFrame(np.zeros([n, 1]), columns=[model.a_idx])
  ax_sample = pd.concat([a, df_sample[model.x_idx]], axis=1)
  cate = np.zeros(n, dtype=np.float32)

  for i in [0, 1]:
    e_y = predict_xgb(model.bst_outcome, ax_sample, df_sample[model.y_idx].dtype)
    cate = cate + (2 * i - 1) * e_y
    ax_sample[model.a_idx] = ax_sample[model.a_idx] + 1
  
  return cate

# The following gets a Monte Carlo estimation of CATE with respect to a model
# where only a subset of the covariates is conditioned on.
#
# It does it by:
#
# 1. Generating a large Monte Carlo sample (as given by `num_simulations`)
#    where we first "delete the edges" from X to A and do uniform sampling
#    of A.
# 2. Fit a XGBoost model for the outcome model only on the columns
#    `x_select` that have been previously chosen. This is the Monte Carlo
#    bit, as the model is an approximation to the (assumed to be know)
#    implied marginal CATE model of `model` with respect to `x_select`,
#    for which we don't have an analytical solution.
# 3. Apply the XGBoost model to the units in `df_sample` with respect o
#    `x_select`

def get_montecarlo_cate(data_eval, x_select, num_simulations, model, use_logistic=False):

  x_eval = data_eval[x_select]
  cate_eval = np.zeros(x_eval.shape[0])
  monte_carlo_sample = simulate_from_controlled_backdoor_model(num_simulations, model)

  for i in [0, 1]:
    sel_train = monte_carlo_sample[model.a_idx] == i 
    x_train = monte_carlo_sample[x_select][sel_train]
    y_train = monte_carlo_sample[model.y_idx][sel_train]

    if use_logistic:
      lgt_montecarlo = LogisticRegression()
      lgt_montecarlo.fit(x_train, y_train)
      e_y = lgt_montecarlo.predict_proba(x_eval)[:, 1]
    else:
      bst_montecarlo = learn_xgb(x_train, y_train) 
      e_y = predict_xgb(bst_montecarlo, x_eval, y_train.dtype)
    cate_eval = cate_eval + (2 * i - 1) * e_y
      
  return cate_eval

# Estimate in-sample cate based on data that is assumed to be conditionally ignorable
# (RCTs/backdoor) using a T-Learner based on XGBoost and only covariate columns `x_select`.

def estimate_cate_tlearner(data, x_select, model):
  
  x_data = data[x_select]
  cate_hat = np.zeros(data.shape[0])
  bst_outcome = [None, None]

  for i in [0, 1]:
    sel_train = data[model.a_idx] == i 
    x_train = data[x_select][sel_train]
    y_train = data[model.y_idx][sel_train]

    bst_outcome[i] = learn_xgb(x_train, y_train)
    e_y = predict_xgb(bst_outcome[i], x_data, y_train.dtype)
    cate_hat = cate_hat + (2 * i - 1) * e_y
      
  return cate_hat, bst_outcome

# Use an outlier detection method to evaluate which points are 
# in the support of the distribution.
# 
# In the output, '1' means in-support, '0' means out of support.
#
# The scores are such that the higher, the more confidence we have about
# that point being in-distribution.
#
# TODO: hyperparameter optimization

def learn_support_classifier(train, test):
  clf = IsolationForest(random_state=0).fit(train)
  predictions = clf.predict(test)
  predictions[predictions == -1] = 0
  score = clf.score_samples(test)
  return predictions, score  

# =================================================================================================================
# =================================================================================================================
# The following focuses on functions for the fully synthetic case, where the random model is based on Gaussian 
# distributions for the covariates, and conditional logistic models for the treatment and outcome.
# =================================================================================================================
# =================================================================================================================

# Class SyntheticBackdoorModel
#
# This stores information for synthetic models, which are based on (correlated) standard Gaussians
# as covariates, with treatment and outcome being binary variables generated by a logistic model.

@dataclass
class SyntheticBackdoorModel(BackdoorModel):
  cov_x: np.array    # Covariance of covariates
  coeff_a: np.array  # Coefficients for logistic model of treatment on covariates
  coeff_y: np.array  # Coefficients for logistic model of outcome of treatment and covariates

# Generate parameters of a SyntheticBackdoorModel.
#
# This is done by sampling the covariance matrix of covariates out of a inverse Wishart
# distribution, where the scale parameter is a correlation matrix with off-diagonal
# elements being given by `rho`. The degrees of freedom are set to a minimum viable
# amount to make the sampling distribution of covariance matrices more diffuse.
#
# We then rescale these covariances to have a diagonal of 1.
#
# Another important input is `prob_positive`, which adds a bias for positive
# coefficients in the model. This is to avoid expected cancellations when `num_x`
# is large so that the total effect of covariates is not accidentally small.

def simulate_synthetic_backdoor_parameters(num_x, rho, prob_positive):
  df = num_x + 1
  shape_mat = np.ones([num_x, num_x]) * rho
  shape_mat[np.diag_indices_from(shape_mat)] = 1    
  cov_x = cov_to_corr(invwishart.rvs(df=df, scale=shape_mat*df))

  coeff_a = np.abs(np.random.normal(loc=0, scale=1/np.sqrt(num_x), size=num_x + 1))
  sign = np.random.uniform(size=num_x + 1)
  coeff_a[sign > prob_positive] = -coeff_a[sign > prob_positive]

  coeff_y = np.abs(np.random.normal(loc=0, scale=1/np.sqrt(num_x + 1), size=num_x + 2))
  sign = np.random.uniform(size=num_x + 2)
  coeff_y[sign > prob_positive] = -coeff_y[sign > prob_positive]
 
  a_idx = 'A'
  x_idx = ['X' + str(i) for i in range(num_x)]
  y_idx = 'Y'

  new_model = SyntheticBackdoorModel(cov_x=cov_x, coeff_a=coeff_a, coeff_y=coeff_y,
                                     x_idx=x_idx, a_idx=a_idx, y_idx=y_idx)
  return new_model

# Simulation from a fitted model.
#
# Basic simulation out of a causal model based on a Gaussian/llogistic fit.
#
# Inputs:
#
# - `n` is the number of samples
# - `model` is a fitted model of the type `SyntheticBackdoorModel`

@dispatch(int, SyntheticBackdoorModel)
def simulate_from_backdoor_model(n, model):
  num_x = model.cov_x.shape[0]
  x = np.random.multivariate_normal(np.zeros(num_x), model.cov_x, n)
  a_prob = logistic(np.concatenate((np.ones([n, 1]), x), axis=1)@model.coeff_a)
  a = np.array(np.random.uniform(0, 1, size=n) <= a_prob, dtype=int).reshape(n, 1)
  y_prob = logistic(np.concatenate((np.ones([n, 1]), x, a), axis=1)@model.coeff_y)
  y = np.array(np.random.uniform(0, 1, size=n) <= y_prob, dtype=int).reshape(n, 1)

  dat = pd.DataFrame(np.concatenate([a, x, y], axis=1), 
                     columns=[model.a_idx] + model.x_idx + [model.y_idx])
  dataframe_categorize(dat, 2)

  return dat

@dispatch(int, SyntheticBackdoorModel)
def simulate_from_controlled_backdoor_model(n, model):
  num_x = len(model.x_idx)
  controlled_model = copy.deepcopy(model)
  controlled_model.coeff_a = np.zeros(num_x + 1) # Erase "edges" from X to A
  dat = simulate_from_backdoor_model(n, controlled_model) 
  return dat

# Selection of observable columns.
#
# This decides which covariates to keep so that everything else will count as unmeasured 
# # confounding. We will the absolute value of the logistic coefficients for that.
#
# Inputs:
#
# - `keep_k`: how many of the covariates to keep as observable.
# - `model`: the corresponding `SyntheticBackdoorModel` model.

@dispatch(int, SyntheticBackdoorModel)
def confounder_column_keeper(keep_k, model):
  keep_x = np.argsort(np.abs(model.coeff_a[1:]))[-keep_k:]
  x_select = [model.x_idx[i] for i in keep_x]
  return x_select

# Compute CATE from given model.
#
# Inputs:
#
# - `df_sample` is the table of covariates we evaluate at
# - `model` is the corresponding `SyntheticBackdoorModel` fitted model.

@dispatch(pd.DataFrame, SyntheticBackdoorModel)
def get_cate(df_sample, model):
  
  n = df_sample.shape[0]
  x = df_sample[model.x_idx]
  a = np.zeros([n, 1])
  cate = np.zeros(n)

  for i in [0, 1]:      
    e_y = logistic(np.concatenate((np.ones([n, 1]), x, a), axis=1)@model.coeff_y)
    cate = cate + (2 * i - 1) * e_y
    a = a + 1
  
  return cate


---

### Example of Application (Based on Real Data)

The following fits real data to some postulated causal structure, and then runs a pipeline of steps to compare how badly unmeasured confounding has affected a simple model fitting procedure.

In [4]:
# EXAMPLE OF APPLICATION
#
# This data is the diabetes data from the UCI repository, available at 
# https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset
#
# The "application" below is totally bogus. I'm using blood pressure as the cause ("A"), and heart disease as
# the outcome ("Y"). Everything else are "pre-treatment" covariates X, and that there is no unmeasured 
# confounding left. Of course there is no reason to believe that either of those assestions are true. 
# The point is just to generate synthetic data from parameters which are not arbitrary.
#
# This made-up example has very weak effects.

cat_threshold = 10 # How many unique values in a column before we decide for it to be continuous
n = 50000 # Large sample size to more easily understand the effect of a wrong model
keep_k = 1 # This is a very sparse model, removing all but one of the covariates for illustration purposes
num_simulations = 100000 # To fit a model for Monte Carlo calculation of true CATE

df_main = pd.read_csv('/Users/ricardosilva/Downloads/diabetes_012_health_indicators_BRFSS2015.csv')
dataframe_categorize(df_main, cat_threshold)
a_idx, y_idx = 'HighBP', 'HeartDiseaseorAttack'
x_idx = list(set(df_main.columns) - set([a_idx, y_idx]))

# Train simulated model and generate synthetic data from fitted model, along with ground truth CATE
print('Fitting ground truth model...')
bst_model = learn_backdoor_model(df_main, x_idx, a_idx, y_idx)
df_sample = simulate_from_backdoor_model(n, bst_model)
true_cate = get_cate(df_sample, bst_model)
print('Fitting Monte Carlo approximation of ground truth CATE with unmeasured confounding...')
x_select = confounder_column_keeper(keep_k, bst_model)
true_cate_xselect = get_montecarlo_cate(df_sample, x_select, num_simulations, bst_model)

# Fit model with all covariates, and a subset of the covariates
print('Fitting causal effect estimators...')
cate_hat_full, _ = estimate_cate_tlearner(df_sample, bst_model.x_idx, bst_model)
cate_hat_xselect, _ = estimate_cate_tlearner(df_sample, x_select, bst_model)

print()

# Please notice that the CATE cases are not directly comparable, as the CATE changes between problems
print('RESULTS FOR SEMI-SYNTHETIC DATA EXPERIMENT')
print('------------------------------------------')
print('Mean absolute CATE error for correct case:', np.mean(abs(true_cate - cate_hat_full)))
print('Mean absolute CATE error for biased case:', np.mean(abs(true_cate_xselect - cate_hat_xselect)))
print('ATE error for correct case:', np.mean(true_cate) - np.mean(cate_hat_full))
print('ATE error for biased case:', np.mean(true_cate_xselect) - np.mean(cate_hat_xselect))

Fitting ground truth model...
Fitting Monte Carlo approximation of ground truth CATE with unmeasured confounding...
Fitting causal effect estimators...

RESULTS FOR SEMI-SYNTHETIC DATA EXPERIMENT
------------------------------------------
Mean absolute CATE error for correct case: 0.024482564080845332
Mean absolute CATE error for biased case: 0.05995307259559631
ATE error for correct case: -0.00827891700429085
ATE error for biased case: -0.05995307259559632


---

### Example of Application (Purely Synthetic Data)

The following sample parameters of a postulated causal structure, and then runs a pipeline of steps to compare how badly unmeasured confounding has affected a simple model fitting procedure.

In [5]:
# EXAMPLE OF APPLICATION
#
# The model is purely synthetic. Check class `SyntheticBackdoorModel` for more information about
# its structure.

num_x = 20          # Number of covariates
rho = 0.2           # Hyperparameter controlling how (positively) correlated covariates are expected to be
prob_positive = 0.7 # Bias for positive coefficients
n = 50000           # Large sample size to more easily understand the effect of a wrong model
keep_k = 1          # This is a very sparse model, removing all but one of the covariates for illustration purposes

# Randomly generate simulated model and then generate synthetic data from fitted model, along with ground truth CATE.
synth_model = simulate_synthetic_backdoor_parameters(num_x, rho, prob_positive)
df_sample_synth = simulate_from_backdoor_model(n, synth_model)
true_cate_synth = get_cate(df_sample_synth, synth_model)
print('Fitting Monte Carlo approximation of ground truth CATE with unmeasured confounding...')
x_select_synth = confounder_column_keeper(keep_k, synth_model)
true_cate_xselect = get_montecarlo_cate(df_sample_synth, x_select_synth, num_simulations, synth_model)

# Fit model with all covariates, and a subset of the covariates. Notice that we
# still use XGBoost to fit models here.
print('Fitting causal effect estimators...')
cate_hat_full_synth, _ = estimate_cate_tlearner(df_sample_synth, synth_model.x_idx, synth_model)
cate_hat_xselect_synth, _ = estimate_cate_tlearner(df_sample_synth, x_select_synth, synth_model)

print()

# Please notice that the CATE cases are not directly comparable, as the CATE changes between problems
print('RESULTS FOR PURELY SYNTHETIC DATA EXPERIMENT')
print('--------------------------------------------')
print('Mean absolute CATE error for correct case:', np.mean(abs(true_cate - cate_hat_full_synth)))
print('Mean absolute CATE error for biased case:', np.mean(abs(true_cate_xselect - cate_hat_xselect_synth)))
print('ATE error for correct case:', np.mean(true_cate) - np.mean(cate_hat_full_synth))
print('ATE error for biased case:', np.mean(true_cate_xselect) - np.mean(cate_hat_xselect_synth))

Fitting Monte Carlo approximation of ground truth CATE with unmeasured confounding...
Fitting causal effect estimators...

RESULTS FOR PURELY SYNTHETIC DATA EXPERIMENT
--------------------------------------------
Mean absolute CATE error for correct case: 0.058161083194382374
Mean absolute CATE error for biased case: 0.09034168918177485
ATE error for correct case: -0.020486986545175313
ATE error for biased case: -0.032672322159260504


---

## Debug Code: How Good is My CATE Estimation?

The following generates synthetic data, then does CATE estimation using a plain T-Learner with correct assumptions of conditional ignorability. With large sample sizes, it should do OK.

Two points to be tested:

* while `get_cate` can get the exact CATE, it only works if we condition on all covariates used to generate the data. In contrast, `get_montecarlo_cate` uses Monte Carlo simulation for when, even when using the true parameters, we focus at a different CATE using a subset of the covariates. This has no analytical formula, hence the simulation. This code tests this implementation.

* we also compare how well our default estimation method, the T-Learner with XGBoost (to be replaced eventually) does in terms of recovering the ground truth CATE

In [ ]:
# HYPERPARAMETERS OF SIMULATION
#

num_x = 20             # Number of covariates
rho = 0.2              # Hyperparameter controlling how (positively) correlated covariates are expected to be
prob_positive = 0.7    # Bias for positive coefficients
n = 500000             # Large sample size to more easily understand the effect of possible bugs
n_simulations = 100000 # Number of Monte Carlo simulations

# GENERATE SYNTHETIC MODEL AND EXACT CATEs
# 

synth_model = simulate_synthetic_backdoor_parameters(num_x, rho, prob_positive) # Model parameters
df_sample_synth = simulate_from_backdoor_model(n, synth_model)                  # Data sampled from model to define where CATE is learned
true_cate_synth = get_cate(df_sample_synth, synth_model)       

# GET MONTECARLO CATE AND COMPARE
#

print('Getting Monte Carlo estimate of true CATE...')
true_cate_synth_montecarlo = get_montecarlo_cate(df_sample_synth, synth_model.x_idx, n_simulations, synth_model)
true_cate_synth_montecarlo_logistic = get_montecarlo_cate(df_sample_synth, synth_model.x_idx, n_simulations, synth_model, use_logistic=True)
print()

print('RESULTS FOR MONTE CARLO EXPERIMENT')
print('----------------------------------')
print('Absolute CATE error for correct model:', np.mean(abs(true_cate_synth - true_cate_synth_montecarlo)))
print('ATE error for correct model:', np.mean(true_cate_synth) - np.mean(true_cate_synth_montecarlo))
print('Rank correlation of exact and Monte Carlo CATEs: ', spearmanr(true_cate_synth, true_cate_synth_montecarlo)[0])
print('Absolute CATE error for correct model (logistic):', np.mean(abs(true_cate_synth - true_cate_synth_montecarlo_logistic)))
print('ATE error for correct model (logistic):', np.mean(true_cate_synth) - np.mean(true_cate_synth_montecarlo_logistic))
print('Rank correlation of exact and Monte Carlo CATEs (logistic): ', spearmanr(true_cate_synth, true_cate_synth_montecarlo_logistic)[0])

print()

# GET T-LEARNER ESTIMATE AND COMPARE

print('Learning causal effect...')
cate_hat, _ = estimate_cate_tlearner(df_sample_synth, synth_model.x_idx, synth_model)
print()

print('RESULTS FOR ESTIMATION EXPERIMENT')
print('---------------------------------')
print('Absolute CATE error for estimated model vs truth:', np.mean(abs(true_cate_synth - cate_hat)))
print('ATE error for estimated model vs truth:', np.mean(true_cate_synth) - np.mean(cate_hat))
print('Rank correlation of true and estimated CATEs: ', spearmanr(true_cate_synth, cate_hat)[0])
print('Absolute CATE error for estimated model vs Monte Carlo:', np.mean(abs(true_cate_synth_montecarlo - cate_hat)))
print('ATE error for estimated model vs Monte Carlo:', np.mean(true_cate_synth_montecarlo) - np.mean(cate_hat))
print('Rank correlation of true (Monte Carlo) and estimated CATEs: ', spearmanr(true_cate_synth_montecarlo, cate_hat)[0])

plt.scatter(true_cate_synth, cate_hat)
plt.xlabel('True CATE')
plt.ylabel('Estimated CATE')


---

## Debug Code: Illustration of Support Detection

In what follows, we will sample from a "RCT" according to some random rejection criterion, then a large observational sample, then
train a classifier to see how observational samples are classified as being in the support of the RCT.

In [ ]:
# First set assorted simulation parameters

num_x = 20              # Number of covariates
rho = 0.2               # Hyperparameter controlling how (positively) correlated covariates are expected to be
prob_positive = 0.7     # Bias for positive coefficients
keep_k = 20             # This indicates how many of the covariates to keep, while treating all others as hidden
max_n_rct = 1000        # Maximum allowed RCT sample
n_obs = 50000           # Observational sample size
n_simulations = 100000  # Number of simulations for a Monte Carlo fitting of the CATE to a subset of covariates

# Generate true causal model, plus an observational sample, and compute their corresponding true CATEs.
# Only covariate columns in x_select are preserved, the rest are hidden (still accessible within full_obs_sample)

ground_truth_model = simulate_synthetic_backdoor_parameters(num_x, rho, prob_positive)
full_obs_sample = simulate_from_backdoor_model(n_obs, ground_truth_model)
x_select = confounder_column_keeper(keep_k, ground_truth_model)
obs_sample = full_obs_sample[[ground_truth_model.a_idx] + x_select + [ground_truth_model.y_idx]]
obs_true_cates = get_montecarlo_cate(obs_sample, x_select, n_simulations, ground_truth_model)
obs_cate_hat, _ = estimate_cate_tlearner(obs_sample, x_select, ground_truth_model)

# What follows works like this: we first sample an unrestricted RCT dataset (of size n_obs).
# We then read a matrix of restrictions which are given by matrix r_A and vector r_b where
# we only keep those rows such that r_A * x <= r_b. 

pre_rct_sample = simulate_from_controlled_backdoor_model(n_obs, ground_truth_model)

num_restrictions = 2
r_A, r_b = np.zeros([num_x + 2, num_restrictions]), np.zeros(num_restrictions)
r_A[np.where(full_obs_sample.columns == 'X1')[0][0], 0], r_b[0] = -1, -1 # X1 >= 1
r_A[np.where(full_obs_sample.columns == 'X2')[0][0], 1], r_b[1] = 1, 0   # X2 <= 0
drop_units = np.where((pre_rct_sample@r_A > r_b).sum(axis=1) > 0)[0]

rct_sample = pre_rct_sample.drop(drop_units).reset_index(drop=True)
if rct_sample.shape[0] > max_n_rct:
  rct_sample = rct_sample.drop(range(max_n_rct, rct_sample.shape[0]))

rct_cate_hat, _ = estimate_cate_tlearner(rct_sample, x_select, ground_truth_model)

# Demonstration of support classifier
in_rct_hat, in_rct_score_hat = learn_support_classifier(rct_sample[x_select], obs_sample[x_select])
in_rct_goldstandard = np.array((full_obs_sample@r_A <= r_b).prod(axis=1))
print('Rank correlation of true and estimated supports: ', spearmanr(in_rct_score_hat, in_rct_goldstandard)[0])
print('Classification error: ', np.mean(in_rct_hat != in_rct_goldstandard))
print('Baseline error "all in sample": ', np.mean(in_rct_goldstandard))
print_confusion_matrix(in_rct_goldstandard, in_rct_hat)

---

## A Learning Pipeline for Matrix-Decomposition Extrapolation with Independent Components

*NOTE: THE FOLLOWING IS STILL WORK IN PROGRESS!*

The below is a library of functions for experimental grounding for a set of units colleted under observational conditions, using another set of units collected under a RCT. This is done by matrix factorization in a matrix containing bias correction terms between the RCT model and the observational one. The main method labels the columns by an increasing set of covariates by which we define relative CATEs. These covariates are not raw covariates, but independent components of the original covariates, so that the ordering can be decided by a simple greedy search.

The main function below is`learn_eta_indep`, which works as follows. Given

* `obs_sample`, the observational sample
* `rct_sample`, the RCT sample
* `x_select`, a list of strings containing the column names of covariates that define our CATE of interest. This is because both the data and the model may contain variables we assume to be either irrelevant, or to be hidden for benchmarking purposes
* `model`, a basic data structure that contains the basics of the causal assumptions - basically, the name of the treatment variablee (`model.a_idx`), the name of the outcome variable (`model.y_idx`) and the list of names of the covariate variables (`model.x_idx`)
* `max_ica_iter`, the maximum number of steps in an independent component analysis method to be called. **This means we assume that covariates are continuous, or with some generosity, binary.**

The way this function works is as follows. First, using `map_to_independent`, it obtains the ICA embedding of the selected covariates, `obs_z` and `rct_z`. These are of type `np.ndarray` i.e. NumPy matrices. We call these independent features $Z$.

Following that, our call to `get_id_support_masks` will run through all $Z$ variables, one at a time, and run an outlier detection method to decide which observational samples are in the marginal support of the corresponding RCT distribution. In particular,

* `in_rct` is a list where `in_rct[j][i]` indicates whether the observation `obs_z[i, j]` is in the support of a distribution with sample `rct_z[:, j]` - in this case `obs_z[i,j] == 1` if the method decides it is the case.
* `in_rct_score[i, j]` is whatever scoring criterion used by the method inside `get_1d_support_masks` to quantify how "likely" `obs_z[i, j]` is under the distribution learned from `rct_z[:, j]`. It is not necessarily a probability, but the higher the more likely so it can be used to rank points and decide other thresholding criteria.
* `score_features[j]` counts many datapoints in `obs_z[:, j]` are in the support, so it's basically `np.sum(obs_z[:, j])`
* `sorted_support` is obtained by sorting `score_features` in a decreasing order, and getting the corresponding indices. So `sorted_support[0]` indicates the feature number (the column number of `obs_z`) with a highest `score_features`, `sorted_support[1]` is a second highest, etc.
* `M[i, k]` is a matrix of observability i.e., it indicates whether sample `obs_z[i, :]` is such that all features `obs_z[i, sorted_support[:k]]` are classified to be in the corresponding RCT supports (as recorded in `in_rct[i, :]`)

Given all that information, we then call `get_cate_and_bias` to obtain

* `tau[i, k]`, CATE estimates for sample `obs_z[i, sorted_support[0:k]]`, where the estimate is given by the model learned using the RCT data
* `omega[i, k]`, CATE estimates for sample `obs_z[i, sorted_support[0:k]]`, where the estimate is given by the model learned using the observational data and corresponding assumptions
* `eta[i, k]`, merely the difference `tau[i, k] - omega[i, k]`

TODO: *the next stage does low-rank matrix completion*

In [ ]:
def learn_eta_indep(obs_sample: pd.DataFrame, rct_sample: pd.DataFrame, 
                    x_select: list, model: BackdoorModel, max_ica_iter=1000):

  # First, get the independent projections  
  obs_z, rct_z = map_to_independent(obs_sample, rct_sample, x_select, max_ica_iter)

  # Get support information:
  # - in_rct[j][i] indicates that sample i is in the support of the RCT distribution
  #   of feature j
  # - in_rct_score[j][i] is a corresponding fitness criterion (the higher, the more
  #   likely point j is in the RCT along dimension Z[j] - but this is not necessarily a probability)
  # - score_features[j] is the number of points in the observational sample with its marginal
  #   over Z[j] also in the support of the RCT
  # - sorted_support[0] indicates which of j = 1, 2, ..., num_selected has the highest
  #   corresponding score_features, sorted_support[1] has the second highest etc.
  # - M[i, j] is 1 if datapoint i is in the support of RCT with respect to Z[sorted_support[0]:sorted_support[j]]
  #   which here, since we are assuming independent features, can be calculated efficiently
  in_rct, in_rct_score, score_features, sorted_support, M = get_1d_support_masks(obs_z, rct_z)

  # Now, run estimators of causal effect by RCT and causal model over th entire sequence
  # of features, without worries about extrapolation
  tau, omega, eta = get_cate_and_bias(obs_z, rct_z, obs_sample, rct_sample, x_select, sorted_support, model)

  # TODO: matrix completion

  return tau, omega, eta, obs_z, rct_z, M
  
def map_to_independent(obs_sample: pd.DataFrame, rct_sample: pd.DataFrame, x_select: list, max_ica_iter=1000):
  # Learn the independent components, Z, according to the observational distribution.
  # Apply this mapping to both the observational data and the RCT sample.
  ica = FastICA(n_components=len(x_select), random_state=0, max_iter=max_ica_iter)
  obs_z = ica.fit_transform(obs_sample[x_select])
  rct_z = ica.transform(rct_sample[x_select])
  return obs_z, rct_z

def get_1d_support_masks(obs_z: np.ndarray, rct_z: np.ndarray):
  # Search for support: classify components one entry at a time to support a greedy search later.
  # Unlike many functions here, ndarrays are used. We are representing features learned from
  # the covariates only.
  #
  # In the below, "in_rct[i]" denotes indicates which observations samples are
  # in the support of the RCT distribution with respect to feature Z[i]. This is
  # a 0/1 decision. In contrast, "in_rct_score[i]" indicates the corresponding
  # measures of fit: the higher, the more "likely" sample i is typical of the
  # RCT distribution.
  # 
  # "score_features[i]" counts how many samples are in the support of the RCT,
  # according to dimension Z[i].
  n, num_select = obs_z.shape[0], obs_z.shape[1]
  in_rct, in_rct_score = [None] * num_select, [None] * num_select
  score_features = np.zeros(num_select, dtype=int)
  for i in range(num_select):
    in_rct[i], in_rct_score[i] = learn_support_classifier(rct_z[:, i:i+1], obs_z[:, i:i+1])
    score_features[i] = sum(in_rct[i])

  # We will now build the support matrix for chosen features.
  #
  # Here, M[i, j] indicates whether observational sample i in the RCT
  # using the first j features according to a given permutation. For
  # the ICA-induced Z features, this is just the order given by
  # score_features_z.

  sorted_support = np.argsort(score_features)[::-1]
  M = np.zeros([n, num_select], dtype=int)
  M[:, 0] = in_rct[sorted_support[0]]
  for j in range(1, num_select):
    next_sel = in_rct[sorted_support[j]]
    M[:, j] = M_z[:, j - 1]
    M[next_sel == 0, j] = 0

  # Now, sort rows to put cases with highest coverage earlier on.
  # Keep track of the original positions.
  #row_scores = np.sum(M, axis=1)
  #new_row_idx = np.argsort(row_scores)[::-1]
  #M = M[new_row_idx, :]

  return in_rct, in_rct_score, score_features, sorted_support, M #, new_row_idx

def get_cate_and_bias(obs_z: np.ndarray, rct_z: np.ndarray, 
                      obs_sample: pd.DataFrame, rct_sample: pd.DataFrame, x_select: list,
                      sorted_support: np.ndarray, model: BackdoorModel):
  
  num_select = len(x_select)
  y_type = obs_sample[model.y_idx].dtype

  # First, create corresponding DataFrames out of featurized data to facilitate calling 
  # existing functions

  z_idx = ['Z' + str(i) for i in range(num_select)]
  rct_z_df = pd.DataFrame(rct_z, columns=z_idx)
  dataframe_categorize(rct_z_df, 10)
  rct_z_df = pd.concat([rct_sample[model.a_idx], rct_sample[model.y_idx], rct_z_df], axis=1)
  obs_z_df = pd.DataFrame(obs_z, columns=z_idx)
  dataframe_categorize(rct_z_df, 10)
  obs_z_df = pd.concat([obs_sample[model.a_idx], obs_sample[model.y_idx], obs_z_df], axis=1)

  rct_z_ce, obs_z_ce = [None] * num_select, [None] * num_select
  tau, omega = np.zeros([n, num_select]), np.zeros([n, num_select])
  for i in range(num_select):
    z_idx_i = [z_idx[j] for j in sorted_support[:i+1]]
    _, rct_z_ce[i] = estimate_cate_tlearner(rct_z_df, z_idx_i, model)
    _, obs_z_ce[i] = estimate_cate_tlearner(obs_z_df, z_idx_i, model)
    obs_z_i = obs_z_df[z_idx_i]
    tau[:, i] = predict_xgb(rct_z_ce[i][1], obs_z_i, y_type) - \
                predict_xgb(rct_z_ce[i][0], obs_z_i, y_type)
    omega[:, i] = predict_xgb(obs_z_ce[i][1], obs_z_i, y_type) - \
                  predict_xgb(obs_z_ce[i][0], obs_z_i, y_type)
    
  eta = tau - omega

  return tau, omega, eta